In [15]:
import pandas as pd
import json
import os

import numpy as np
import faiss
from langchain.docstore.document import Document
from langchain.document_loaders import JSONLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI

from dotenv import load_dotenv

load_dotenv()

True

In [16]:
def load_bias_terms(file_path: str) -> list[Document]:

    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    documents = []
    for entry in data:
        bias_type = entry.get("Bias_Type", "Unknown Bias")
        description = entry.get("Description", "")
        content = f"Bias: {bias_type}\nDescription: {description}"
        documents.append(Document(page_content=content, metadata={"type":"definition", "source": file_path}))
    
    return documents

def load_document_text(file_path: str) -> list[Document]:
    
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    documents = []
    for entry in data:
        country = entry.get("Country", "Unknown country")
        doc_name = entry.get("Document Name", "")
        text_chunk = entry.get("Text", "")
        content = f"Country: {country}\nDocument name: {doc_name}\nSentence: {text_chunk}"
        documents.append(Document(page_content=content, metadata={"type":"text", "source": file_path}))
    return documents

def load_document_metrics(file_path: str) -> list[Document]:
    
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    documents = []
    for entry in data:
        country = entry.get("Country", "Unknown country")
        doc_name = entry.get("Document Name", "")
        bias_type = entry.get("Most Prevalent Bias", "")
        content = f"Country: {country}\nDocument Name: {doc_name}\nMost Prevalent Bias: {bias_type}"

        prevalence_score = entry.get("Prevalence Score", "")
        mean_similarity_score = entry.get("Mean Similarity Score", "")
        bias_frequency = entry.get("Bias Frequency", "")
        mean_tfidf_score = entry.get("Mean Normalized TF-IDF", "")
        metadata = {"type":"bias type", "Prevalence Score":prevalence_score, "Mean Similarity Score":mean_similarity_score, "Bias Frequency":bias_frequency, 
                    "Mean Normalized TF-IDF":mean_tfidf_score, "source": file_path}

        documents.append(Document(page_content=content, metadata=metadata))
    return documents


bias_documents = load_bias_terms("../RAG_data/bias_terms.json")
text_documents = load_document_text("../RAG_data/documents_text.json")
document_metrics = load_document_metrics("../RAG_data/document_metrics.json")

documents = bias_documents + text_documents + document_metrics
print(f"Loaded {len(documents)} documents.")
print(documents[34843])

Loaded 43039 documents.
page_content='Country: THA
Document name: 12th National Economic and Social Development Plan
Sentence: 7 Develop management system s and conflict resolution mechanisms  regarding natural resource s and environment al issues : 3' metadata={'type': 'text', 'source': '../RAG_data/documents_text.json'}


In [17]:
print(document_metrics[0])

page_content='Country: ARG
Document Name: Laura
Most Prevalent Bias: Racial bias' metadata={'type': 'bias type', 'Prevalence Score': 21.393770405, 'Mean Similarity Score': 0.2863502733, 'Bias Frequency': 260, 'Mean Normalized TF-IDF': 0.2873534269, 'source': '../RAG_data/document_metrics.json'}


In [18]:
def filter_by_country(country_code: str, documents: list[Document]) -> list[Document]:
    return [
        doc for doc in documents
        if country_code in doc.metadata.get("Country", "") or country_code in doc.page_content
    ]

def make_country_filtered_retriever(base_retriever, country_code: str):
    def get_relevant_documents(query: str) -> list[Document]:
        all_docs = base_retriever.get_relevant_documents(query)
        return filter_by_country(country_code, all_docs)
    return get_relevant_documents

In [19]:
import os
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI 

# Initialize embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/LaBSE")

index_path = "../../Data/faiss_labse_index"

if os.path.exists(index_path):
    vectorstore = FAISS.load_local(index_path, embeddings, allow_dangerous_deserialization=True)
    print("Vectorstre loaded")
else:
    print("Creating vector store")
    vectorstore = FAISS.from_documents(documents, embeddings)
    vectorstore.save_local(index_path)

retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

all_documents: list[Document] = vectorstore.docstore._dict.values()


def retrieve_country_specific_docs(query: str, country_code: str, k: int = 5) -> list[Document]:
    text_docs_filtered = [
        doc for doc in all_documents
        if doc.metadata.get("type") == "text" and country_code in doc.page_content
    ]

    other_docs = [
        doc for doc in all_documents
        if doc.metadata.get("type") != "text"
    ]

    combined_docs = text_docs_filtered + other_docs

    temp_index = FAISS.from_documents(combined_docs, embeddings)
    return temp_index.similarity_search(query, k=k)

# gpt-4o-2024-08-06
# Initialize your LLM using the updated ChatOpenAI for chat-based models

llm = ChatOpenAI(temperature=0, model_name="gpt-4o-2024-08-06", openai_api_key=os.environ["OPENAI_API_KEY"])

# Build the RetrievalQA (RAG) chain
qa_chain = RetrievalQA.from_chain_type(llm, chain_type="stuff", retriever=retriever)


Vectorstre loaded


In [20]:
docs = retrieve_country_specific_docs("What is the most prevalent bias in Denmark?", " DNK", k=5)
for i, doc in enumerate(docs):
    print(f"\n--- Document {i} ---\n{doc.page_content}")


--- Document 0 ---
Country: DNK
Document name: Priority Areas
Sentence: Today, IT specialists constitute just over 4% of the employed in Denmark

--- Document 1 ---
Country: DNK
Document name: Priority Areas
Sentence: This is a problem for Danish competitiveness, knowledge-development, and future digitisation of Denmark  in businesses, in the public sector and in research

--- Document 2 ---
Country: DNK
Document name: Priority Areas
Sentence: Denmark has high priority on research

--- Document 3 ---
Country: DNK
Document name: Priority Areas
Sentence: More investment in Danish businesses3

--- Document 4 ---
Country: DNK
Document name: Priority Areas
Sentence: Denmark is one of the worlds most digitised countries


In [21]:
#vectorstore.save_local("../faiss_labse_index")

In [22]:
# Query the RAG system using the invoke method
query = "What is the description of Confirmation bias?"
result = qa_chain.invoke(query)
print(result)


{'query': 'What is the description of Confirmation bias?', 'result': 'The description of Confirmation bias is the search for and use of information to support an individual’s ideas, beliefs, or hypotheses.'}


In [23]:
# Query the RAG system using the invoke method
query = "What is the most prevalent bias from USA documents? explain why"
result = qa_chain.invoke(query)
print(result)

{'query': 'What is the most prevalent bias from USA documents? explain why', 'result': 'The most prevalent bias from the USA documents provided is racial bias. This is evident from the documents "AI and Society" and "Fairness, Ethics, Accountability, and Transparency," both of which identify racial bias as the most prevalent. Racial bias is a significant concern in AI and societal contexts because it can lead to unfair treatment and discrimination against individuals based on their race, which is why it is a focal point in discussions about fairness, ethics, and accountability in AI systems.'}


In [24]:
# Query the RAG system using the invoke method
query = "What is the most prevalent bias in DNK?"
result = qa_chain.invoke(query)
print(result)

{'query': 'What is the most prevalent bias in DNK?', 'result': 'The most prevalent bias in Denmark (DNK) is popularity bias.'}


In [25]:
docs = retriever.get_relevant_documents(query)
for i, doc in enumerate(docs):
    print(f"\n--- Document {i} ---\n{doc.page_content}")


--- Document 0 ---
Country: DNK
Document Name: Priority Areas
Most Prevalent Bias: Popularity bias

--- Document 1 ---
Country: DNK
Document name: Priority Areas
Sentence: Finally, in an international context, the Danish population is well-educated, almost all Danes use the internet on a daily basis, and there is a high degree of mutual trust and confidence

--- Document 2 ---
Country: DNK
Document name: Priority Areas
Sentence: Today, IT specialists constitute just over 4% of the employed in Denmark

--- Document 3 ---
Country: DNK
Document name: Priority Areas
Sentence: Common Danish language resource2

--- Document 4 ---
Country: DNK
Document name: Priority Areas
Sentence: Denmark has high priority on research


In [26]:
ground_truth = [
    {
        "country": "Canada",
        "language": "English",
        "abbreviation": " CAN",
        "question": "What have Canadian Indigenous researchers and experts from around the world teamed up to publish?",
        "answers": [
            "ca/ai/ai-society14TH20TH24THAICan Impact Report 202022CENTRING INDIGENOUS PERSPECTIVES IN DESIGNING AIIndigenous researchers and experts from around the world have teamed up to publish Indigenous Protocol and Artificial Intelligence, a position paper32 that provides a starting place for designing ethical AI through an Indigenous-centred approach"
        ]
    },
    {
        "country": "Denmark",
        "language": "English",
        "abbreviation": " DNK",
        "question": "What role does Carlsberg play in Denmark’s national AI strategy, and how is AI used in their beer production process?",
        "answers": [
            "Artificial intelligence will help beer tasting at Carlsberg1,000 different beers are screened every day at the Carlsberg laboratory in Valby near Copenhagen"
        ]
    },
    {
        "country": "Spain",
        "language": "Spanish",
        "abbreviation": " ESP",
        "question": "What evidence does the Spanish strategy provide to demonstrate that Spain has a higher percentage of female researchers in AI compared to the EU average?",
        "answers": [
            "España ya cuenta con un mayor número de mujeres investigadores, el 38,8% frente al 33,8% de la UE 25, según datos del INE y Eurostat, lo que signiﬁca una buena posición para impulsar la reducción de esta brecha de género"
        ]
    },
    {
        "country": "European Union",
        "language": "English",
        "abbreviation": " EU",
        "question": "What limitations does the report identify in the EU’s use of experimental policymaking, such as regulatory sandboxes?",
        "answers": [
            "Assessment of current initiatives of the European Commission on better regulation Third, despite the plethora of tools available to Commission services in the Toolbox, the use of experimental policymaking in the form of regulatory sandboxes and similar instruments remain orphan of a general framework at the EU level, which would allow Member States to engage in experimental policymaking",
            "As it stands, Tool #69 on “Emerging Methods and Policy Instruments”, however laudable, risks providing only a theoretical opportunity for Commission policymakers, rooted in the possibility to include provisions for sandboxes in the legislative texts, rather than engaging in experimentation themselves, or relying on a structured process of experimentation in Member States while proposals are still in the making"
        ]
    },
    {
        "country": "Italy",
        "language": "Italian",
        "abbreviation": " ITA",
        "question": "What strategy does the Italian government propose to reduce the innovation gap between northern and southern regions of the country using AI?",
        "answers": [
            "La rete nazionale per lIA sar inter regionale, e permetter ai territori di scambiarsi la conoscenza e le competenze per rimanere al passo dell innovazione dell IA, favorire ladozione di queste tecnologie da parte delle PMI, e quindi r idurre il divario nell'innovazione, nello sviluppo e nella competitivit delle imprese tra Nord e Sud"
        ]
    },
    {
        "country": "Netherlands",
        "language": "Dutch",
        "abbreviation": " NLD",
        "question": "What concrete measures does the Dutch strategy propose to double the reuse of public data for AI development within five years?",
        "answers": [
            "30 AI VOOR NEDERLANDVERGROTEN,VERSNELLEN EN VERBINDEN 31    1,0 0,8 0,4 0,6 0,2 0,00,240,180,23NederlandVerenigde statenOostenrijkMexicoJapanFrankrijkSpanjeColombia Nieuw-ZeelandDuitslandFinlandAustralieCanadaZuid-KoreaIsraelIerland GriekenlandNoorwegen ScoreItalie   Overheid stimuleert hergebruikData BeschikbaarheidData ToegankelijkheidOm de bruikbaarheid van publieke data te verbete-ren, moet data worden opgeschoond, gelabeld en gekoppeld",
            "Daarnaast moet er een duidelijk verdienmodel komen waarin de kosten voor de overheid in ver-houding zijn met de baten voor de directe gebrui-ker en de maatschappij",
            "Over vijf jaar bereiken we hopelijk een verdubbeling van het hergebruik van publieke data"
        ]
    },
    {
        "country": "Sweden",
        "language": "English",
        "abbreviation": " SWE",
        "question": "According to the Vinnova report, what factors mutually reinforce or inhibit the development of AI-based business and operational models in Sweden?",
        "answers": [
            "Business and operational models, data access and competence are mutually dependent and therefore heavily affected by each other in companies and public operations",
            "Without clear prospects of business benefits, the drivers of AI-based investments are inhibited",
            "If the business benefit is not clear, AI competence is also not viewed as an important factor for value creation and efficiency, affecting recruitment patterns and competence development",
            "Data access and possibilities of combining different data will be fundamentally significant for purposes of identifying the applications that can be developed"
        ]
    },
    {
        "country": "United States",
        "language": "English",
        "abbreviation": " USA",
        "question": "Why does the U.S. National AI R&D Strategic Plan stress the importance of developing AI systems that can explain their decisions?",
        "answers": [
            "First, Strategy 4 emphasizes the need for explainable and transparent systems that are trusted by their users , perform in a manner that is acceptable to the users, and can be guaranteed to act as the user intended",
            "Thus, research ers must develop systems that are transparent, and intrinsically capable of explaining the reasons  for their results to users"
        ]
    }
]


In [27]:
def evaluate_retriever(ground_truth_data, retriever, top_k: int = 5):
    import pandas as pd
    results = []
    total_hits = 0
    total_recall = 0

    for item in ground_truth_data:
        question = item["question"]
        expected_answers = item["answers"]
        country = item["country"]
        language = item["language"]
        country_code = item["abbreviation"]

        retrieved_docs = retrieve_country_specific_docs(question, country_code)
        retrieved_texts = [doc.page_content.lower() for doc in retrieved_docs]
        expected_texts = [ans.lower() for ans in expected_answers]
        hit = any(
            any(expected in retrieved for retrieved in retrieved_texts)
            for expected in expected_texts
        )
        total_hits += int(hit)

        relevant_found = sum(
            any(expected in retrieved for retrieved in retrieved_texts)
            for expected in expected_texts
        )
        recall = relevant_found / len(expected_texts)
        total_recall += recall

        results.append({
            "Country": country,
            "Language": language,
            "Hit Score": 1 if hit else 0,
            "Recall": round(recall, 2)
        })

    # Add average
    avg_hit = round(total_hits / len(ground_truth_data), 2)
    avg_recall = round(total_recall / len(ground_truth_data), 2)
    results.append({
        "Country": "AVERAGE",
        "Language": "",
        "Hit Score": avg_hit,
        "Recall": avg_recall
    })

    df = pd.DataFrame(results)
    print(df.to_string(index=False))
    return df

In [28]:
df_results = evaluate_retriever(ground_truth, retriever, top_k=5)

       Country Language  Hit Score  Recall
        Canada  English       1.00    1.00
       Denmark  English       1.00    1.00
         Spain  Spanish       1.00    1.00
European Union  English       0.00    0.00
         Italy  Italian       1.00    1.00
   Netherlands    Dutch       1.00    0.33
        Sweden  English       0.00    0.00
 United States  English       1.00    0.50
       AVERAGE                0.75    0.60


In [29]:
df_results

,Country,Language,Hit Score,Recall
0,Canada,English,1.00,1.00
1,Denmark,English,1.00,1.00
2,Spain,Spanish,1.00,1.00
3,European Union,English,0.00,0.00
4,Italy,Italian,1.00,1.00
5,Netherlands,Dutch,1.00,0.33
6,Sweden,English,0.00,0.00
7,United States,English,1.00,0.50
8,AVERAGE,,0.75,0.60
